In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
dataset = pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [11]:
dataset = pd.get_dummies(dataset,dtype=int, drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [13]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [15]:
independent = dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [17]:
dependent = dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [19]:
#Splitting Train and Test Set
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent, dependent, test_size=0.30, random_state=0)

In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

#Here the fit.transform function Standardize the Undefined data to Defined Data and make it to be saved in X_train
X_train = sc.fit_transform(X_train)

#Here the transform function uses the process occurred in fit.transform function for the input X_test and make it to be saved in X_train
X_test = sc.transform(X_test)

X_train

array([[-1.5330973 , -0.40713453, -0.89833872, -0.97676557, -0.50466988],
       [-0.03364163,  0.32855417, -0.89833872, -0.97676557, -0.50466988],
       [ 0.89459283,  2.56690911,  3.25603402, -0.97676557, -0.50466988],
       ...,
       [ 0.03776102, -0.91016269, -0.89833872,  1.02378711, -0.50466988],
       [-1.46169465,  0.76659782, -0.89833872,  1.02378711, -0.50466988],
       [-0.46205754, -1.96596021, -0.06746417, -0.97676557, -0.50466988]])

In [23]:
from sklearn.model_selection import GridSearchCV #Here cv refers to Cross Validation
from sklearn.svm import SVR

#Creating a Dictionary for Playable Parameters
param_grid = {'kernel':['rbf','poly','sigmoid','linear'], 'C':[10,100,1000,2000,3000], 'gamma':['auto','scale']}

#if refit = false then final model will be considered as Last Combination of parameter_grid like (linear,3000,scale)
#n_jobs = -1 refers to Run all the Models Parallely
grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3, n_jobs = -1)

#All the above steps are called "Procedure Load"

#Model Creation
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             verbose=3)

In [24]:
#print Best Parameter after Tuning the Playable Parameters
#print(grid.best_params_)

#To get all the results of GridSearchCV
results = grid.cv_results_
#print(results)

print("The R_Score Value for the Best Parameter {}:".format(grid.best_params_))

The R_Score Value for the Best Parameter {'C': 3000, 'gamma': 'scale', 'kernel': 'poly'}:


In [25]:
#To make All the Results into Tabular Format

table = pd.DataFrame.from_dict(results)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.071923,0.011629,0.036573,0.004572,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.004055,0.013366,-0.103821,-0.095119,-0.101604,-0.056625,0.053504,35
1,0.053202,0.003068,0.008119,0.001844,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.056274,0.069532,-0.045601,-0.025079,-0.049592,0.001107,0.051309,32
2,0.070658,0.010703,0.012978,0.006151,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.049905,0.075905,-0.046585,-0.041004,-0.046507,-0.001657,0.053391,34
3,0.047571,0.005754,0.008763,0.003808,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.377969,0.479601,0.317872,0.337979,0.324422,0.367569,0.059777,25
4,0.059161,0.003809,0.031320,0.004145,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.004126,0.013244,-0.103775,-0.095165,-0.101602,-0.056634,0.053486,36
5,0.046482,0.002700,0.006575,0.003009,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.054964,0.071297,-0.046513,-0.024157,-0.049652,0.001188,0.051594,31
6,0.065266,0.004976,0.014911,0.004806,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.049644,0.076323,-0.046798,-0.040824,-0.046521,-0.001635,0.053474,33
7,0.048264,0.005165,0.008286,0.004666,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.377969,0.479601,0.317872,0.337979,0.324422,0.367569,0.059777,25
8,0.056084,0.002911,0.030273,0.001120,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.300573,0.339474,0.173708,0.217991,0.183375,0.243024,0.065733,29
9,0.050673,0.008892,0.012289,0.004678,100,auto,poly,"{'C': 100, 'gamma': 'auto', 'kernel': 'poly'}",0.540643,0.575051,0.474839,0.535730,0.424640,0.510181,0.053582,21


In [26]:
age_input = float(input("Age:"))
bmi_input = float(input("BMI:"))
children_input = float(input("Children:"))
gender_male_input = float(input("Gender Male 0 or 1:"))
smoker_yes_input = float(input("Smoker Yes 0 or 1:"))

Age: 32
BMI: 40
Children: 2
Gender Male 0 or 1: 1
Smoker Yes 0 or 1: 0


In [31]:
#prediction
#Keyword - predict is function of Application 
Future_Prediction = grid.predict([[age_input, bmi_input, children_input, gender_male_input, smoker_yes_input]])
print("Future_Prediction{}:".format(Future_Prediction))

Future_Prediction[3950978.03598634]:


In [ ]:
7

In [33]:
# Naming the Model with its extension .sav
import pickle
filename = "finalized_model_SVM_Regression_GridSearch_Assignment.sav"

#Model will be saved in respective path, once after the below Code Execution
# Keyword - wb refers to "write binary"
pickle.dump(grid,open(filename,'wb'))

In [37]:
# Load the Saved Model and check the same to send it for Phase-02 (Deployment)
# # Keyword - rb refers to "read binary"

loaded_model=pickle.load(open("finalized_model_SVM_Regression_GridSearch_Assignment.sav",'rb'))
result = loaded_model.predict([[age_input, bmi_input, children_input, gender_male_input, smoker_yes_input]]) 
result

array([3950978.03598634])